In [1]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.0 MB/s eta 0:00:00


In [2]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.Random import get_random_bytes
import base64
import os

In [3]:
def generate_and_save_rsa(bits=2048, priv_path="private.pem", pub_path="public.pem", passphrase=None):
    key = RSA.generate(bits)
    priv_pem = key.export_key(passphrase=passphrase, pkcs=8) if passphrase else key.export_key()
    pub_pem = key.publickey().export_key()
    with open(priv_path, "wb") as f:
        f.write(priv_pem)
    with open(pub_path, "wb") as f:
        f.write(pub_pem)
    print(f"Generated RSA {bits}-bit. Saved -> {priv_path}, {pub_path}")
    return priv_pem, pub_pem

# contoh panggilan
priv_pem, pub_pem = generate_and_save_rsa(2048)

Generated RSA 2048-bit. Saved -> private.pem, public.pem


In [5]:
def rsa_encrypt(pub_pem: bytes, message: bytes) -> str:
    pub = RSA.import_key(pub_pem)
    cipher = PKCS1_OAEP.new(pub)
    ct = cipher.encrypt(message)
    return base64.b64encode(ct).decode()

def rsa_decrypt(priv_pem: bytes, b64_ciphertext: str) -> bytes:
    priv = RSA.import_key(priv_pem)
    cipher = PKCS1_OAEP.new(priv)
    ct = base64.b64decode(b64_ciphertext)
    return cipher.decrypt(ct)

# contoh penggunaan
msg = b"Hello Cryptography - Nabilla & Muthi"
ct_b64 = rsa_encrypt(pub_pem, msg)
print("Ciphertext (base64):", ct_b64)
plaintext = rsa_decrypt(priv_pem, ct_b64)
print("Decrypted plaintext:", plaintext.decode())

Ciphertext (base64): PTqqeJDEylpdoMdy7q4i6E+oRD9anehFOBfSr/5c6LDHahH4GhDCcpZSxj36WZn+0dYvkqbNC/0w6PIkDrA3ZaiWVo/DsobTcdxRB4kMT5nsjoTUWfUA2Le0blnaYJLOTz7xDgd2Bb5AS/BG3Olc7+yFZqICIvfujheO+hgyQ7a1PgR8ktFC5PUsE8wi7BH8xK2lAI1VGTvegjUzeSDO/4pCr0Y3hSyzwzZJw6JhzkEk3XRaBqPZBXv5o49IrXzg4unPyxHBXKl9Qce7tEWvX6DU5HkHqxC8kKH83QnzhvaiQaJqdFLyWxZxKEK9NrP1YwqbIpubAf07iC58LtpI7w==
Decrypted plaintext: Hello Cryptography - Nabilla & Muthi


In [6]:
def hash_sha256_bytes(data: bytes) -> SHA256.SHA256Hash:
    h = SHA256.new(data=data)
    return h

# contoh
h = hash_sha256_bytes(b"Message to sign")
print("SHA-256 hexdigest:", h.hexdigest())

SHA-256 hexdigest: 1351934e68754024164bd423dd77d26dfd5307b09d36250da950ed3cf976e5c5


In [10]:
def rsa_sign_pss(priv_pem: bytes, data: bytes) -> str:
    priv = RSA.import_key(priv_pem)
    h = SHA256.new(data=data)
    signer = pss.new(priv)
    signature = signer.sign(h)
    return base64.b64encode(signature).decode()

def rsa_verify_pss(pub_pem: bytes, data: bytes, b64_signature: str) -> bool:
    pub = RSA.import_key(pub_pem)
    h = SHA256.new(data=data)
    signature = base64.b64decode(b64_signature)
    verifier = pss.new(pub)
    try:
        verifier.verify(h, signature)
        return True
    except (ValueError, TypeError):
        return False

# contoh
to_sign = b"Dokumen penting - Nabilla & Muthi"
sig_b64 = rsa_sign_pss(priv_pem, to_sign)
print("Signature (base64):", sig_b64)
print("Verify (original):", rsa_verify_pss(pub_pem, to_sign, sig_b64))
print("Verify (tampered):", rsa_verify_pss(pub_pem, to_sign + b"!", sig_b64))

Signature (base64): N7+vfhhqhVZySjWOJoa9CeWbDlayk97Evr4xU/fLzbBp6t/6oK1otwpe1Y4RGVNiUeuUNVH3hGKlWN9EiOH10RIhzQntuZkFhzdaSlA1UuHq1E2NWFCBzo9bgDH0aylNClLOwoNvVqLbsuDPjoENtWkluITojKVN6YqWalot9ZpM9kM0oBnS5ZMLurXZGh4MflZ0Ug+3PsovRxCarwnkVlmj4rgnA5R7kBTmsHEk9erZZLRuvlxS6Yft7XjYMEh+LPUcxzpyKAlDozQVZNEh2r5145rx53sW6hZLsv4FDKKYK4hXeYu/QOa9AiiDD5xsAARQ7QeuhsbYS8cberfPqw==
Verify (original): True
Verify (tampered): False


In [8]:
original = b"Report: Tugas 2 RSA"
sig = rsa_sign_pss(priv_pem, original)
print("Verify original:", rsa_verify_pss(pub_pem, original, sig))
tampered = original[:-1] + b"?"
print("Verify tampered:", rsa_verify_pss(pub_pem, tampered, sig))

Verify original: True
Verify tampered: False


In [9]:
with open("private.pem","rb") as f: priv_pem = f.read()
with open("public.pem","rb") as f: pub_pem = f.read()